In [2]:
import pandas as pd
import re
import matplotlib.pyplot as plt
from datetime import timedelta
import numpy as np
import seaborn

In [3]:
coagulation_raw = pd.read_excel("blood coagulation.xlsx")

In [4]:
patients = pd.read_csv("ids.csv")
ids = patients["id"].tolist()
print(f"Total number of patients: {len(ids)}")

Total number of patients: 1204


In [5]:
coagulation_raw = coagulation_raw[coagulation_raw["Рег.№"].isin(ids)]
coagulation = coagulation_raw[["Рег.№", "Возр.", "Дата забора", "Результаты"]].reset_index()
coagulation = coagulation.dropna(subset = ["Результаты"])

def filter_test_results(t_res):
    t_res = str(t_res)
    return re.sub(r"\d*\.?\d+", "", t_res)

In [6]:
test_list  = coagulation["Результаты"].apply(filter_test_results).unique().tolist()
unique_tests = []
for test in test_list:
    tmp = test.split(":")
    for t in tmp:
        if t not in unique_tests:
            unique_tests.append(t)

unique_tests = ["APTT", "PT", "INR", "FBG", "% PT", "TT", "AT%", "D-dimer"]

In [7]:
coagulation["Результаты"] = coagulation["Результаты"].str.split(";")
coagulation = coagulation.explode("Результаты")

In [8]:
def get_test_values(test):
    # print(test)
    res = []
    test = str(test)
    test = test.replace(" ", "")
    tmp = test.split(":")
    if tmp[1] == "":
        return []
    # print(tmp)
    for t in unique_tests:
        if t in tmp[0]:
            res.append(t)
            # print(res)
            break
    if not len(res) == 1:
        return []
    res.append(re.search(r"\d*\.?\d+", tmp[1])[0])
    # print(res)
    return res

In [9]:
coagulation["Результаты"] = coagulation["Результаты"].apply(get_test_values)

In [ ]:
coagulation[["Test", "Value"]] = coagulation["Результаты"].apply(pd.Series)
coagulation = coagulation[["Рег.№", "Дата забора", "Test", "Value"]]
coagulation.to_excel("coagulation_parsed.xlsx")